<a href="https://colab.research.google.com/github/DFS-33/Limpeza_dados_python/blob/main/cliente_planilha_Douglas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import copy

### ETL Utilizando Pandas

Este é um dos cenários que eu tive experiência na última empresa a qual eu trabalhei. Em alguns casos a fonte de dados dos clientes, a qual eu  teria que fazer as migrações eram planilhas de cadastros ao invés de banco de dados. 

Exemplo: Cadastro de clientes, Serviços, produtos, profissionais.

Diante deste cenário, optei por fazer essas alterações utilizando a linguagem Python e pacote Pandas.<br> 


<b>Processo: Consiste em ler as planilhas, analisar, transformar os dados e carregar posteriormente em um banco de dados relacional (Mysql)<b>




In [ ]:
# Lendo planilha de clientes enviada e salvando o dataframe em uma variável
df = pd.read_excel('/content/clientes.xlsx')

In [ ]:
# Verificando a quantidade linhas e colunas 
df.shape

(1449, 13)

In [ ]:
# Para padronização vamos deixar todas as colunas em minusculo
df.columns = df.columns.str.lower()

In [ ]:
df.head()

,salao_id,nome,cpf,datanasc,e-mail,celular,endereco,numero,complemento,unnamed: 9,unnamed: 10,cep,sexo
0,76865,Abadia,NaN,30/11/-0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76865,ABGAIL FELIPE DIAS,038.812.176-90,2000-08-01 00:00:00,NaN,(31) 98388-0909,Rua Osvaldo Lima e Silva,1264,NaN,Belo Horizonte,MG,30626-670,Feminino
2,76865,ABGAIR MOREIRA CAMPOS,417.616.696-72,1939-09-11 00:00:00,NaN,(31) 9961-25323,Rua Joaquim Aristóteles de Oliveira,54,NaN,Belo Horizonte,MG,31570-090,Feminino
3,76865,ADEISA CASSIA DRUMONT LOPES COLEN,NaN,30/11/-0001,NaN,(31) 9915-98237,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,76865,Adelaide Ferreira Cardoso,564.062.316-00,30/11/-0001,abgaircampos@ibest.com.br,(31) 9843-02815,Rua Eugênio Sales,560,NaN,Belo Horizonte,MG,31550-240,Feminino


In [ ]:
# Renomeando nome das colunas do dataframe, para o mesmo nome das colunas da base de dados a serem migradas as informações posteriormente.
df.rename(columns={'codigo_cpf':'cpf',
                   'fone1':'celular',
                   'dtnasc':'datanasc',
                   'fone2':'telefone'}, inplace=True)

In [ ]:
# Selecionando apenas as variáveis que de fato vamos migrar para a nossa base de dados
df = df[['nome','cpf','celular','telefone','endereco','bairro','obs','cep','cidade','uf', 'datanasc','numero']]

In [ ]:
# Formatando o nome como title 
df['nome'] = df.nome.str.title()

In [ ]:
# Lendo os primeiros registros
df.head()

,salao_id,nome,cpf,datanasc,e-mail,celular,endereco,numero,complemento,unnamed: 9,unnamed: 10,cep,sexo
0,76865,Abadia,NaN,30/11/-0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76865,Abgail Felipe Dias,038.812.176-90,2000-08-01 00:00:00,NaN,(31) 98388-0909,Rua Osvaldo Lima e Silva,1264,NaN,Belo Horizonte,MG,30626-670,Feminino
2,76865,Abgair Moreira Campos,417.616.696-72,1939-09-11 00:00:00,NaN,(31) 9961-25323,Rua Joaquim Aristóteles de Oliveira,54,NaN,Belo Horizonte,MG,31570-090,Feminino
3,76865,Adeisa Cassia Drumont Lopes Colen,NaN,30/11/-0001,NaN,(31) 9915-98237,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,76865,Adelaide Ferreira Cardoso,564.062.316-00,30/11/-0001,abgaircampos@ibest.com.br,(31) 9843-02815,Rua Eugênio Sales,560,NaN,Belo Horizonte,MG,31550-240,Feminino


### Tratando variaveis numericas
Para não ter incosistência no momento de inserir as informações no banco de dados e visualização na aplicação é necessário fazer alguns tratamentos.

- coluna datananasc formato (yyyy/mm/dd)
- coluna celular (retirar caracteres especiais exemplo (31) 98388-0909
- coluna cep (retirar caracteres 31570-090)

In [ ]:

# função para retirar caracteres especiais
import re 
def corrigir_celular(cel):
    new_cel = re.sub(r"[\D]","",cel)
    return new_cel

In [ ]:
# Aplicando função para retirar caracteres especiais na coluna celular
df['celular'] = df['celular'].fillna('').apply(str) 
df['celular'] = df['celular'].apply(corrigir_celular)

In [ ]:
# Aplicando função para retirar caracteres especiais na coluna cep
df['cep'] = df['cep'].fillna('').apply(str) 
df['cep'] = df['cep'].apply(corrigir_celular)

In [ ]:
# Aplicando função para retirar caracteres especiais na coluna cpf
df['cpf'] = df['cpf'].fillna('').apply(str) 
df['cpf'] = df['cpf'].apply(corrigir_celular)

In [ ]:
# Lendo os primeiros registros
df.head()

,salao_id,nome,cpf,datanasc,e-mail,celular,endereco,numero,complemento,unnamed: 9,unnamed: 10,cep,sexo
0,76865,Abadia,,30/11/-0001,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN
1,76865,Abgail Felipe Dias,03881217690,2000-08-01 00:00:00,NaN,31983880909,Rua Osvaldo Lima e Silva,1264,NaN,Belo Horizonte,MG,30626670,F
2,76865,Abgair Moreira Campos,41761669672,1939-09-11 00:00:00,NaN,31996125323,Rua Joaquim Aristóteles de Oliveira,54,NaN,Belo Horizonte,MG,31570090,F
3,76865,Adeisa Cassia Drumont Lopes Colen,,30/11/-0001,NaN,31991598237,NaN,NaN,NaN,NaN,NaN,,NaN
4,76865,Adelaide Ferreira Cardoso,56406231600,30/11/-0001,abgaircampos@ibest.com.br,31984302815,Rua Eugênio Sales,560,NaN,Belo Horizonte,MG,31550240,F


In [ ]:
# Verificando os valores unicos da coluna sexo
df.sexo.unique()

array([nan, 'Feminino', 'Masculino', '---'], dtype=object)

In [ ]:
# Tratando a coluna sexo 
df.loc[df.sexo == 'Masculino','sexo'] = 'M'
df.loc[df.sexo == 'Feminino','sexo'] = 'F'
df.loc[df.sexo == 'feminino','sexo'] = 'F'
df.loc[df.sexo == '---','sexo'] = ''
df.loc[df.sexo == ' ','sexo'] = ''

In [ ]:
# Adicionando colunas necessárias no dataframe para posterior importação na base de dados

df['salao_id']= 80524  
df['cliente']= 1
df['responsavel'] = 'Douglas Fernando'

In [ ]:
# Verificando se existem registros duplicados no cadastro de clientes

df[['nome','celular','endereco','datanasc']].duplicated().sum()

# Nenhum registro encontrado

0

In [ ]:
# Verificando tamanho 
df.shape

(1449, 13)

In [ ]:
# Salvando dataframe formatado em excel, pronto para ser inserido na base de dado
df.to_excel('/content/clientes_1449.xlsx', index=False)